# PHS564 — Lecture 10 (Student)
## Causal survival analysis: time-to-event outcomes, censoring, discrete-time hazards (MIMIC-IV Demo)

### Learning goals
- Distinguish **risk** vs **hazard** vs **survival**; know what effect measure you are estimating.
- Convert a cohort into **person-period** (discrete time) data and fit pooled logistic hazards.
- Handle **right censoring** with **IPCW**; understand when censoring is “informative”.
- Produce causal survival curves under a point treatment (and interpret assumptions).

### Required reading
- Hernán & Robins, sections on censoring and survival (target trial chapters/sections as applicable).

**Rules for this notebook**
- Only edit cells marked **TODO**.
- Do not change the overall structure/cell order.


In [ ]:
# Colab bootstrap (run this first if you opened from a Colab badge)
# - Clones the repo into /content/PHS564 (if needed)
# - Installs requirements
# - Adds repo to sys.path

from __future__ import annotations

import os
import sys
import subprocess
from pathlib import Path


def _in_colab() -> bool:
    return "google.colab" in sys.modules


if _in_colab():
    REPO_URL = "https://github.com/vafaei-ar/PHS564.git"
    TARGET_DIR = Path("/content/PHS564")

    if not (TARGET_DIR / "requirements.txt").exists():
        print("Cloning course repo into Colab runtime...")
        subprocess.run(["git", "clone", "--depth", "1", REPO_URL, str(TARGET_DIR)], check=True)

    os.chdir(TARGET_DIR)

    print("Installing requirements...")
    subprocess.run([sys.executable, "-m", "pip", "-q", "install", "-r", "requirements.txt"], check=True)

    if str(TARGET_DIR) not in sys.path:
        sys.path.insert(0, str(TARGET_DIR))

    print("✓ Colab setup complete. Now run the rest of the notebook.")
else:
    print("Not running in Colab; skipping Colab bootstrap.")


### Setup

This notebook is designed to run **locally** or in **Google Colab**.

**If you opened from the Colab badge (recommended):**
1) Run the first code cell titled **“Colab bootstrap”** (it clones the repo + installs requirements)
2) Run the notebook top-to-bottom.

**If you are running locally:**
- Install dependencies from `requirements.txt` (see the repo `README.md`), then run top-to-bottom.


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reproducibility
RNG = np.random.default_rng(564)

# Locate repo root (works when running from lectures/Lxx.../student or /instructor)
THIS_DIR = Path.cwd()
REPO_ROOT = THIS_DIR
for _ in range(4):
    if (REPO_ROOT / "requirements.txt").exists() or (REPO_ROOT / "README.md").exists():
        break
    REPO_ROOT = REPO_ROOT.parent

DATA_DIR = REPO_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"

print("Working directory:", THIS_DIR)
print("Repo root:", REPO_ROOT)
print("Processed data dir exists:", PROC_DIR.exists())


### Build the processed cohort extract (required)

This lecture expects an analysis-ready cohort file in `data/processed/`.

If it’s missing, run the next cell to:
1) download MIMIC-IV Demo into `data/raw/` (if needed)
2) build the processed cohort extracts into `data/processed/`

**Exposure mode (A):** default uses `admission_type` (contains “EMER”). You can switch to `vitals` by changing `EXPOSURE_MODE` in the code cell.

In [ ]:
# Build processed cohort extracts from raw MIMIC-IV Demo (safe to re-run)
# This will create cohort files under data/processed/.

EXPOSURE_MODE = "admission_type"  # or "vitals"
HR_THRESHOLD = 100.0  # used only if EXPOSURE_MODE == "vitals"

try:
    from data.download_data import download_mimic_demo
    from data.build_processed_extracts_demo import build_processed_extracts

    mimic_dir = RAW_DIR / "mimic-iv-demo-2.2"
    if not mimic_dir.exists():
        print("Downloading raw MIMIC-IV Demo (v2.2) to data/raw/ ...")
        download_mimic_demo(out_dir=RAW_DIR, version="2.2", method="python")
    else:
        print("✓ Raw MIMIC-IV Demo already present.")

    out_paths = build_processed_extracts(exposure_mode=EXPOSURE_MODE, hr_threshold=HR_THRESHOLD)
    print("✓ Built processed cohorts:")
    for k, v in out_paths.items():
        print(f"  {k}: {v}")
except Exception as e:
    print("Could not build processed cohort extracts in this environment.")
    print("Error:", e)
    print("If you already have the cohort file, place it in data/processed/ and re-run.")

### Optional: download raw MIMIC-IV Demo tables

Not required for the homework pipeline. Skip unless your instructor asks you to explore the raw Demo tables in `data/raw/`.

In [ ]:
# Optional: explore raw tables (safe to skip)
# The cohort-build cell above already downloads what you need for this lecture.

try:
    from data.download_data import download_mimic_demo

    download_mimic_demo(out_dir=RAW_DIR, version="2.2", method="python")
except Exception as e:
    print("Skipping raw MIMIC-IV Demo download.")
    print("Error:", e)

## Data
This lecture uses a **processed cohort extract** derived from MIMIC-IV Demo.

Expected file: `data/processed/cohort_L10_survival.parquet` (or `.csv`).

If this file is missing:
- Run the “Build the processed cohort extract” cell above, or
- Run locally: `python data/build_processed_extracts_demo.py --exposure-mode admission_type`

Assumed columns (you may adapt to the actual extract):
- `A` treatment at baseline
- `T` follow-up time (in discrete intervals)
- `E` event indicator (1=event, 0=censored)
- baseline covariates (e.g., age, sex, severity)

We will fit a **discrete-time hazard** model via logistic regression on a person-period dataset.


### Column definitions (this cohort)

These are defined by the cohort builder (`data/build_processed_extracts_demo.py`):

- `A` (baseline treatment): `A = 1` if `admission_type` contains `"EMER"` (e.g., "EW EMER.", "DIRECT EMER.")
- `E` (event): `E = 1` if the patient **died in-hospital** (`hospital_expire_flag`)
- `T` (follow-up time, days): `T = ceil(dischtime - admittime)` with administrative truncation at **28 days**


### Before you load the cohort (important)

If you **do not** already have `data/processed/cohort_L10_survival.parquet` (or `.csv`):
- In Colab: run the cell titled **“Build the processed cohort extract (required)”** above, then come back here.
- Locally: run `python data/build_processed_extracts_demo.py --exposure-mode admission_type`.


In [ ]:
# Statsmodels for regression (logit/ols); installed via requirements.txt
import statsmodels.api as sm
import statsmodels.formula.api as smf

parquet_path = PROC_DIR / "cohort_L10_survival.parquet"
csv_path = PROC_DIR / "cohort_L10_survival.csv"

if parquet_path.exists():
    df = pd.read_parquet(parquet_path)
elif csv_path.exists():
    df = pd.read_csv(csv_path)
else:
    raise FileNotFoundError(
        "Missing cohort file for L10. Build it via:\n"
        "  - Colab: run the build cell above\n"
        "  - Local: python data/build_processed_extracts_demo.py --exposure-mode admission_type\n"
    )

df.head()

### TODO A1 — Create person-period dataset
If your extract is already long-format (one row per person-period), skip this.
Otherwise, expand each subject into rows `t=1..T` and set event indicator at the event time.


In [ ]:
# Our built cohort is *wide* with columns:
#   hadm_id, A, E (event), T (follow-up time in days), age, sex, ...
# We will expand it into a simple person-day dataset with:
#   t = 1..T, and E_t = 1 at the event time if E==1.

if "sex_male" not in df.columns:
    df["sex_male"] = (df["sex"].astype(str).str.upper() == "M").astype(int)

records = []
for _, r in df[["hadm_id", "A", "E", "T", "age", "sex_male"]].dropna().iterrows():
    T = int(r["T"])
    E = int(r["E"])
    for t in range(1, T + 1):
        E_t = 1 if (E == 1 and t == T) else 0
        records.append(
            {
                "hadm_id": int(r["hadm_id"]),
                "t": t,
                "A": int(r["A"]),
                "E_t": E_t,
                "age": float(r["age"]),
                "sex_male": int(r["sex_male"]),
            }
        )

long = pd.DataFrame.from_records(records)
long.head()

## Part B — Discrete-time hazard model
We model Pr(E_t=1 | E_{t-1}=0, A, L, t).


In [ ]:
# Outcome and covariates (match the person-day dataset we created)
# We include time as categorical (C(t)) to allow a flexible baseline hazard.
hazard_formula = "E_t ~ A + C(t) + age + sex_male"

haz_model = smf.logit(hazard_formula, data=long).fit(disp=False)
haz_model.params.head()

### TODO B1 — Predict survival curves under A=1 and A=0 (g-formula)
Compute survival S(t) = Π_{k<=t} (1 - h(k)).


In [ ]:
def survival_curve(data_long: pd.DataFrame, A_value: int, t_col: str = "t") -> pd.DataFrame:
    # data_long contains one row per person-period and baseline covariates.
    d = data_long.copy()
    d["A"] = A_value
    d["haz"] = haz_model.predict(d)
    # average hazard per time
    hz = d.groupby(t_col)["haz"].mean().sort_index()
    surv = (1 - hz).cumprod()
    return pd.DataFrame({"t": hz.index, "haz": hz.values, "surv": surv.values})

# TODO: adapt t column name if needed
curve1 = survival_curve(long, 1)
curve0 = survival_curve(long, 0)
curve1.head(), curve0.head()

In [ ]:
plt.figure()
plt.step(curve0["t"], curve0["surv"], where="post", label="A=0")
plt.step(curve1["t"], curve1["surv"], where="post", label="A=1")
plt.xlabel("time interval")
plt.ylabel("Survival S(t)")
plt.legend()
plt.title("Estimated survival curves (discrete-time)")
plt.show()

## Reflection
1) Why is time-to-event analysis central to target trial emulation?
2) What is the difference between hazard and risk, and why does it matter?
